# SentenceTransformers SBERT

https://www.sbert.net/index.html


## getting started

In [1]:
import numpy as np
import pandas as pd
import pickle

from sentence_transformers import SentenceTransformer, util

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
## Mult-lingual model that supports Dutch
modelML = SentenceTransformer('xlm-r-100langs-bert-base-nli-mean-tokens')

## import kamer speeches

In [ ]:
## put data chuncks in one zip file
!cat data/CorpusTweedeKamer* > CorpusTweedeKamer.zip

In [2]:
%%time
tweede_kamer = pd.read_csv("CorpusTweedeKamer.zip")
tweede_kamer = (
    tweede_kamer
    .assign(datum = pd.to_datetime(tweede_kamer.date))
    .assign(speaker = tweede_kamer.speaker.str.lower())
)

CPU times: user 13.5 s, sys: 1.09 s, total: 14.6 s
Wall time: 14.9 s


In [3]:
tweede_kamer.shape

(1143366, 12)

In [4]:
recente_speeches = (
    tweede_kamer
    .dropna(subset = ["text"])
    .query('terms > 3')
    .query('terms < 800')
    .query('datum > "2016-01-01"')
)

In [5]:
recente_speeches.shape

(216470, 12)

It takes hours to embed 100K texts on a comouter without GPU. you can spin up a AI VM machine on GCP to speed thing sup



In [ ]:
## if you have gcloud CLI setup: here is code to spin up a machine....

export IMAGE_FAMILY="pytorch-latest-gpu"
export ZONE="europe-west4-a"
export INSTANCE_NAME="lhl-gpu-instance-2"
export INSTANCE_TYPE="n1-standard-8"

gcloud compute instances create $INSTANCE_NAME \
        --zone=$ZONE \
        --image-family=$IMAGE_FAMILY \
        --image-project=deeplearning-platform-release \
        --maintenance-policy=TERMINATE \
        --accelerator="type=nvidia-tesla-v100,count=1" \
        --machine-type=$INSTANCE_TYPE \
        --boot-disk-size=120GB \
        --metadata="install-nvidia-driver=True" \
        --preemptible

# Dont forget to install the right versiuon of torch in our case with the GCP machine we need_slice

pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# and the package sentence-transformers
pip install -U sentence-transformers

In [10]:
speeches = recente_speeches.text.str.lower().values.tolist()

In [77]:
%%time 
sentence_embeddings = modelML.encode(speeches)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


In [62]:
sentence_embeddings.shape

(95314, 768)

In [12]:
#Store sentences & embeddings on disc
with open('kamer_BERT_embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': speeches, 'embeddings': sentence_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)


## use pre computed embedings

In [3]:
#Load sentences & embeddings from disc
with open('kamer_BERT_embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    speeches = stored_data['sentences']
    sentence_embeddings = stored_data['embeddings']

### Voer een semantic search uit

In [19]:
tt = """
We zijn een klein land van 17 miljoen mensen, dat groot is geworden door
samenwerking in onze gezamenlijke strijd tegen het water, door onze handelsgeest
en onze openheid naar de wereld en door onze gemeenschapszin, die mensen
hier een veilig thuis bood. Ons land is al eeuwenlang een plek waar mensen
met verschillende religieuze en culturele achtergronden en botsende politieke
overtuigingen vreedzaam met elkaar samenleven. Waar democratie en recht altijd
het laatste woord hebben.
Het zijn precies die waarden en tradities die nu onder druk staan. De crises van
de afgelopen jaren hebben veel Nederlanders bezorgd en onzeker gemaakt, door
het verlies aan zekerheid, de vrees voor terreur en een toenemend gebrek aan
respect en saamhorigheid. In de ik-samenleving van vandaag hebben we onvoldoende besef van het belang van gedeelde waarden en de onderlinge spelregels
die nodig zijn om met elkaar samen te leven. We zijn te veel gaan geloven in een
vrijheid zonder verantwoordelijkheid, in een wereld van rechten zonder plichten.
De voorbeelden zien we dagelijks om ons heen. Op internet en social media is de
vrijheid van meningsuiting voor sommigen vooral een vrijbrief voor beledigen en
kwetsen. Op straat tarten en treiteren jongeren uit verveling toevallige passanten
en agenten. Maar het gebrek aan waarden en normen zien we ook bij roekeloze
bankiers, sjoemelende multinationals of graaiende bestuurders, die het eigenbelang
boven het algemeen belang stelden. Of bij politici die bewust te grote beloftes doen
om vier jaar later doodleuk sorry te zeggen.
Ondertussen roept de moeizame integratie van 
"""

In [20]:
query_embedding = modelML.encode(tt, convert_to_tensor=True)
topmatches = util.semantic_search(query_embedding, sentence_embeddings)
topmatches

[[{'corpus_id': 83016, 'score': 0.85005826},
  {'corpus_id': 70980, 'score': 0.8331502},
  {'corpus_id': 93926, 'score': 0.8251599},
  {'corpus_id': 19519, 'score': 0.82501054},
  {'corpus_id': 24841, 'score': 0.82154727},
  {'corpus_id': 9857, 'score': 0.81382555},
  {'corpus_id': 46585, 'score': 0.81361353},
  {'corpus_id': 68347, 'score': 0.8127924},
  {'corpus_id': 22523, 'score': 0.80401015},
  {'corpus_id': 62094, 'score': 0.80095875}]]

In [21]:
top_textst = [ idx['corpus_id']  for idx in topmatches[0]]
[speeches[i] for i in top_textst]

['voorzitter. nederland. neder-land: laag land. we wonen sinds mensenheugenis in deze prachtige delta. die delta heeft ons veel gebracht: vervoer via rivieren, handel, grondstoffen, makkelijke verbindingen met andere steden en bescherming tegen de vijand, maar ook overlast en overstromingen. de batavieren wisten het al. en later maakten de nederlanders ons land veilig en zorgden voor dijken en voor droge voeten. we pasten ons altijd aan om hier te mogen leven. veel van onze voorouders streden dus al tegen dat water of profiteerden ervan. tot voorbeeld strekken ook de achternamen in deze kamer. heel veel mensen hebben net als ik "dijk" in hun achternaam. als we hier hoofdelijk stemmen hoor je altijd: van dijk, dijksma, dijkhoff et cetera. waterbeheer zit in onze genen, wil ik zeggen. we beschermen wat we samen hebben opgebouwd, soms polderend, maar ons altijd realiserend wat de kracht en de kwetsbaarheid van het water is. die relatie met het water, dat is er eentje die heel spannend is.

In [22]:
recente_speeches.iloc[top_textst]

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,datum
1110991,2019-02-13,NaN,480,de heer remco dijkstra,VVD,828.0,False,689,Voorzitter. Nederland. Neder-land: laag land. ...,NL-TweedeKamer,NLD,2019-02-13
1078948,2018-10-10,NaN,645,mevrouw diertens,D66,45.0,False,459,"Dank u wel, voorzitter. Het duurt altijd even ...",NL-TweedeKamer,NLD,2018-10-10
1139332,2019-06-26,NaN,186,mevrouw karabulut,SP,1363.0,False,108,"Zeker. Ik ben echt heel begripvol, ook voor uw...",NL-TweedeKamer,NLD,2019-06-26
951132,2016-09-21,NaN,375,de heer zijlstra,VVD,828.0,False,498,Ik denk dat iedereen in deze zaal zich ook dit...,NL-TweedeKamer,NLD,2016-09-21
962594,2016-11-09,NaN,534,de heer martin bosma,PVV,298.0,False,170,"Ik win altijd van de heer Voordewind, voorzitt...",NL-TweedeKamer,NLD,2016-11-09
930110,2016-04-14,NaN,403,de heer graus,PVV,298.0,False,227,U wel? Ik niet. Normaal is hij altijd heel rus...,NL-TweedeKamer,NLD,2016-04-14
1013544,2017-11-22,NaN,234,de heer bosman,VVD,828.0,False,544,"Voorzitter, dank. Elke dag zetten onze mannen ...",NL-TweedeKamer,NLD,2017-11-22
1072275,2018-09-19,NaN,356,de heer dijkhoff,VVD,828.0,False,117,Wij komen al jaren in actie. In de tegenvallen...,NL-TweedeKamer,NLD,2018-09-19
957714,2016-10-25,NaN,261,de heer verhoeven,D66,45.0,False,205,"Inderdaad, het valt in de heer Monasch echt te...",NL-TweedeKamer,NLD,2016-10-25
1055448,2018-05-30,NaN,351,de heer van haga,VVD,828.0,False,771,"Dank u wel, voorzitter. Dit is mijn eerste deb...",NL-TweedeKamer,NLD,2018-05-30


In [36]:
test = sentence_embeddings

In [37]:
%%time
out = util.semantic_search(test,sentence_embeddings)

CPU times: user 13min 13s, sys: 22.1 s, total: 13min 35s
Wall time: 3min 26s


In [31]:
out

[[{'corpus_id': 0, 'score': 1.0000002},
  {'corpus_id': 9838, 'score': 0.93183213},
  {'corpus_id': 38806, 'score': 0.9303728},
  {'corpus_id': 16450, 'score': 0.930214},
  {'corpus_id': 47607, 'score': 0.9261825},
  {'corpus_id': 59494, 'score': 0.92431724},
  {'corpus_id': 75573, 'score': 0.92092705},
  {'corpus_id': 12931, 'score': 0.9189229},
  {'corpus_id': 20641, 'score': 0.91309655},
  {'corpus_id': 11929, 'score': 0.91289246}],
 [{'corpus_id': 1, 'score': 1.0000007},
  {'corpus_id': 33722, 'score': 0.89955425},
  {'corpus_id': 52536, 'score': 0.8993991},
  {'corpus_id': 81220, 'score': 0.8956955},
  {'corpus_id': 66605, 'score': 0.8939451},
  {'corpus_id': 66644, 'score': 0.8939451},
  {'corpus_id': 79398, 'score': 0.89389014},
  {'corpus_id': 7619, 'score': 0.89387417},
  {'corpus_id': 17093, 'score': 0.8916458},
  {'corpus_id': 17075, 'score': 0.8916458}],
 [{'corpus_id': 2, 'score': 1.0000007},
  {'corpus_id': 15512, 'score': 0.92138505},
  {'corpus_id': 21919, 'score': 0.90